In [4]:
import pandas as pd
import re
import phonenumbers
from phonenumbers import carrier
from phonenumbers.phonenumberutil import number_type
from datetime import datetime

phoneFrame = pd.read_csv('dirty.csv', usecols=['contact_phone'])
phoneFrame.to_csv('phone_output.txt', sep='\t', index=False)

def validate_non_empty(validation_function, input_value):
    # Проверяем, не является ли строка пустой
    if not input_value.strip():  # strip() удаляет пробелы в начале и в конце
        return False
    else:
        # Если строка не пустая, выполняем валидацию
        return validation_function(input_value)

def date_validation(client_bday):
    # Регулярное выражение для проверки формата даты в формате 'YYYY-MM-DD'
    date_regex = r'^\d{4}-\d{2}-\d{2}$'

    # Проверка на соответствие регулярному выражению
    if re.match(date_regex, client_bday):
        try:
            # Попробуем преобразовать строку в объект datetime
            datetime.strptime(client_bday, '%Y-%m-%d')
            return True  # Если дата валидна
        except ValueError:
            return False  # Если дата не валидна (например, 2024-02-30)
    else:
        return False  # Если формат не соответствует

def phone_validation(phoneNumber):
    try:
        # Пробуем разобрать номер телефона
        number2Validate = phoneNumber
        # Проверяем, является ли номер мобильным
        if carrier._is_mobile(number_type(phonenumbers.parse(number2Validate))):
            return True  # Номер телефона валиден и мобильный
        else:
            return False  # Номер телефона валиден, но не мобильный
    except phonenumbers.NumberParseException:
        return False  # Невалидный номер телефона

def email_validation(email):
    if isinstance(email, str):  # Проверяем, является ли email строкой
        # Регулярное выражение для проверки формата электронной почты
        email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        # Проверяем, соответствует ли email регулярному выражению
        return re.match(email_regex, email) is not None
    return False  # Если email не строка, возвращаем False

def zip_validation(zip_code):
    if isinstance(zip_code, str):  # Проверяем, является ли zip_code строкой
        # Регулярное выражение для проверки формата почтового индекса
        zip_regex = r'^\d{6}$'  # Формат: 123456
        # Проверяем, соответствует ли zip_code формату
        return re.match(zip_regex, zip_code) is not None
    return False  # Если zip_code не строка, возвращаем False

validationFunctions = {
    'phone_validate': phone_validation,
    'email_validate': email_validation,
    'zip_validate': zip_validation,
    'date_validate': date_validation
}

input_phone = ''  # Пример телефона
is_valid_phone = validate_non_empty(validationFunctions['phone_validate'], input_phone)
print(is_valid_phone)  # Выведет True или False в зависимости от результата

# client_bplace = pd.read_csv('dirty.csv', usecols=['client_bplace'])
# client_bplace.to_csv('res.txt', sep='\t', index=False)

#print(validationFunctions['phone_validate']('+12345678901'))

# # Загружаем данные
# df = pd.read_csv("dirty.csv", low_memory=False)

# # Извлекаем нужные столбцы в goldenRecord
# goldenRecord = df[["client_id", "client_first_name", "client_middle_name", "client_last_name", "client_fio_full", 
#                    "client_bday", "client_bplace", "client_cityzen", "client_resident_cd", "client_gender", 
#                    "client_marital_cd", "client_graduate", "client_child_cnt", "client_mil_cd", "client_zagran_cd", 
#                    "client_inn", "client_snils", "client_vip_cd", "contact_vc", "contact_tg", "contact_other", 
#                    "contact_email", "contact_phone", "addr_region", "addr_country", "addr_zip", "addr_street", 
#                    "addr_house", "addr_body", "addr_flat", "addr_area", "addr_loc", "addr_city", "addr_reg_dt", 
#                    "addr_str", "fin_rating", "fin_loan_limit", "fin_loan_value", "fin_loan_debt", "fin_loan_percent", 
#                    "fin_loan_begin_dt", "fin_loan_end_dt", "stream_favorite_show", "stream_duration", "create_date", 
#                    "update_date", "source_cd"]]

# # Преобразуем столбец update_date в datetime (если это необходимо)
# goldenRecord['update_date'] = pd.to_datetime(goldenRecord['update_date'], errors='coerce')

# # Сортируем данные по дате (сначала актуальные, затем старые)
# goldenRecord = goldenRecord.sort_values(by='update_date', ascending=False)

# # Множество для хранения уникальных СНИЛС
# seen_snils = {}

# # Открываем новый CSV-файл для записи данных с уникальными СНИЛС и актуальными записями
# with open('unique_snils.csv', 'w', newline='', encoding='utf-8') as f:
#     # Записываем заголовки
#     f.write(','.join(goldenRecord.columns) + '\n')  # Используем запятую для заголовков
    
#     # Итерация по строкам DataFrame
#     for index, row in goldenRecord.iterrows():
#         snils = str(row['client_snils'])  # Получаем СНИЛС в виде строки
        
#         # Если СНИЛС еще не встречался, записываем его в файл
#         if snils not in seen_snils:
#             seen_snils[snils] = row['update_date']  # Сохраняем дату последней записи для этого СНИЛСа
            
#             # Заменяем NaN на пустую строку
#             row = row.fillna('')  # Заменяем все NaN на пустую строку
            
#             # Записываем данные через запятую
#             f.write(','.join(row.astype(str)) + '\n')
#         else:
#             # Если СНИЛС уже встречался, проверяем актуальность данных
#             last_update = seen_snils[snils]  # Дата последней записи для этого СНИЛСа
            
#             # Если текущая запись более актуальная (с более поздней датой), обновляем данные
#             if row['update_date'] > last_update:
#                 seen_snils[snils] = row['update_date']  # Обновляем дату для СНИЛСа
                
#                 # Заменяем NaN на пустую строку
#                 row = row.fillna('')  # Заменяем все NaN на пустую строку
                
#                 # Записываем данные через запятую
#                 f.write(','.join(row.astype(str)) + '\n')

# # Уведомление о завершении записи
# print("Данные с уникальными и актуальными СНИЛС были сохранены в файл 'unique_snils.csv'.")


False
